In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
sample_submission = pd.read_csv('./sample_submission.csv')

In [3]:
train.head()

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40


In [4]:
test.head()

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%)
0,1_20220825 00,1,20220825 00,23.5,0.0,2.2,72
1,1_20220825 01,1,20220825 01,23.0,0.0,0.9,72
2,1_20220825 02,1,20220825 02,22.7,0.0,1.5,75
3,1_20220825 03,1,20220825 03,22.1,0.0,1.3,78
4,1_20220825 04,1,20220825 04,21.8,0.0,1.0,77


In [5]:
sample_submission.head()

,num_date_time,answer
0,1_20220825 00,0
1,1_20220825 01,0
2,1_20220825 02,0
3,1_20220825 03,0
4,1_20220825 04,0


In [6]:
train.columns

Index(['num_date_time', '건물번호', '일시', '기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)',
       '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)'],
      dtype='object')

In [7]:
train.shape

(204000, 10)

In [8]:
train['일사(MJ/m2)'].isnull().sum()

87913

In [9]:
train['강수량(mm)'].isnull().sum()

160069

In [10]:
train['일조(hr)'].isnull().sum()

75182

In [11]:
building = pd.read_csv('./building_info.csv')

In [12]:
building

,건물번호,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1,건물기타,110634.00,39570.00,-,-,-
1,2,건물기타,122233.47,99000.00,-,-,-
2,3,건물기타,171243.00,113950.00,40,-,-
3,4,건물기타,74312.98,34419.62,60,-,-
4,5,건물기타,205884.00,150000.00,-,2557,1000
...,...,...,...,...,...,...,...
95,96,호텔및리조트,93314.00,60500.00,-,-,-
96,97,호텔및리조트,55144.67,25880.00,-,-,-
97,98,호텔및리조트,53578.62,17373.75,-,-,-
98,99,호텔및리조트,53499.00,40636.00,-,-,-


In [13]:
# 데이터 전처리
train.head()

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40


In [14]:
train.isnull().sum()

num_date_time         0
건물번호                  0
일시                    0
기온(C)                 0
강수량(mm)          160069
풍속(m/s)              19
습도(%)                 9
일조(hr)            75182
일사(MJ/m2)         87913
전력소비량(kWh)            0
dtype: int64

In [15]:
# NaN인 것은 0으로 처리
train = train.fillna(0)

In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204000 entries, 0 to 203999
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   num_date_time  204000 non-null  object 
 1   건물번호           204000 non-null  int64  
 2   일시             204000 non-null  object 
 3   기온(C)          204000 non-null  float64
 4   강수량(mm)        204000 non-null  float64
 5   풍속(m/s)        204000 non-null  float64
 6   습도(%)          204000 non-null  float64
 7   일조(hr)         204000 non-null  float64
 8   일사(MJ/m2)      204000 non-null  float64
 9   전력소비량(kWh)     204000 non-null  float64
dtypes: float64(7), int64(1), object(2)
memory usage: 15.6+ MB


In [17]:
# 일시 : 시간으로 데이터타입 변경
train['일시'] = pd.to_datetime(train['일시'])

In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204000 entries, 0 to 203999
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   num_date_time  204000 non-null  object        
 1   건물번호           204000 non-null  int64         
 2   일시             204000 non-null  datetime64[ns]
 3   기온(C)          204000 non-null  float64       
 4   강수량(mm)        204000 non-null  float64       
 5   풍속(m/s)        204000 non-null  float64       
 6   습도(%)          204000 non-null  float64       
 7   일조(hr)         204000 non-null  float64       
 8   일사(MJ/m2)      204000 non-null  float64       
 9   전력소비량(kWh)     204000 non-null  float64       
dtypes: datetime64[ns](1), float64(7), int64(1), object(1)
memory usage: 15.6+ MB


In [19]:
#num_date_time열은 건물번호와 일시 열로 정보 반영가능하므로 제거해도 될듯
train = train.drop(['num_date_time'], axis=1)

In [20]:
train

,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1,2022-06-01 00:00:00,18.6,0.0,0.9,42.0,0.0,0.0,1085.28
1,1,2022-06-01 01:00:00,18.0,0.0,1.1,45.0,0.0,0.0,1047.36
2,1,2022-06-01 02:00:00,17.7,0.0,1.5,45.0,0.0,0.0,974.88
3,1,2022-06-01 03:00:00,16.7,0.0,1.4,48.0,0.0,0.0,953.76
4,1,2022-06-01 04:00:00,18.4,0.0,2.8,43.0,0.0,0.0,986.40
...,...,...,...,...,...,...,...,...,...
203995,100,2022-08-24 19:00:00,23.1,0.0,0.9,86.0,0.5,0.0,881.04
203996,100,2022-08-24 20:00:00,22.4,0.0,1.3,86.0,0.0,0.0,798.96
203997,100,2022-08-24 21:00:00,21.3,0.0,1.0,92.0,0.0,0.0,825.12
203998,100,2022-08-24 22:00:00,21.0,0.0,0.3,94.0,0.0,0.0,640.08


In [21]:
# train은 데이터 전처리 완료
# test도 동일한 데이터 전처리 적용
test.head()

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%)
0,1_20220825 00,1,20220825 00,23.5,0.0,2.2,72
1,1_20220825 01,1,20220825 01,23.0,0.0,0.9,72
2,1_20220825 02,1,20220825 02,22.7,0.0,1.5,75
3,1_20220825 03,1,20220825 03,22.1,0.0,1.3,78
4,1_20220825 04,1,20220825 04,21.8,0.0,1.0,77


In [22]:
test['일시'] = pd.to_datetime(test['일시'])
test = test.drop(['num_date_time'], axis=1)

In [23]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16800 entries, 0 to 16799
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   건물번호     16800 non-null  int64         
 1   일시       16800 non-null  datetime64[ns]
 2   기온(C)    16800 non-null  float64       
 3   강수량(mm)  16800 non-null  float64       
 4   풍속(m/s)  16800 non-null  float64       
 5   습도(%)    16800 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(2)
memory usage: 787.6 KB


In [24]:
test.isnull().sum()

건물번호       0
일시         0
기온(C)      0
강수량(mm)    0
풍속(m/s)    0
습도(%)      0
dtype: int64

In [25]:
building.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   건물번호          100 non-null    int64  
 1   건물유형          100 non-null    object 
 2   연면적(m2)       100 non-null    float64
 3   냉방면적(m2)      100 non-null    float64
 4   태양광용량(kW)     100 non-null    object 
 5   ESS저장용량(kWh)  100 non-null    object 
 6   PCS용량(kW)     100 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 5.6+ KB


In [26]:
building.head()
# ess : 남는 전력을 따로 저장했다가 필요시에 공급하는 형태
# pcs : 전력변환장치

,건물번호,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1,건물기타,110634.00,39570.00,-,-,-
1,2,건물기타,122233.47,99000.00,-,-,-
2,3,건물기타,171243.00,113950.00,40,-,-
3,4,건물기타,74312.98,34419.62,60,-,-
4,5,건물기타,205884.00,150000.00,-,2557,1000


In [27]:
building['태양광용량(kW)'].value_counts()
# -는 0으로 바꾸자

-         64
25         2
20         2
50         2
200        2
28.44      1
3          1
9.45       1
365        1
14.4       1
65         1
95         1
60.45      1
621        1
100        1
97.85      1
98.28      1
46.2       1
35         1
100.56     1
40         1
83.72      1
30         1
26.64      1
97.65      1
110        1
70         1
64.8       1
56         1
130.6      1
447.2      1
60         1
21.6       1
Name: 태양광용량(kW), dtype: int64

In [28]:
building['ESS저장용량(kWh)'].value_counts()

-        95
2557      1
800       1
1362      1
837.1     1
3100      1
Name: ESS저장용량(kWh), dtype: int64

In [29]:
building['PCS용량(kW)'].value_counts()

-       95
500      2
1000     1
300      1
800      1
Name: PCS용량(kW), dtype: int64

In [30]:
building['태양광용량(kW)'][building['태양광용량(kW)']=='-']=0

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_29372\3784538752.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  building['태양광용량(kW)'][building['태양광용량(kW)']=='-']=0


In [31]:
building['태양광용량(kW)'].value_counts()

0         64
25         2
20         2
50         2
200        2
28.44      1
3          1
9.45       1
365        1
14.4       1
65         1
95         1
60.45      1
621        1
100        1
97.85      1
98.28      1
46.2       1
35         1
100.56     1
40         1
83.72      1
30         1
26.64      1
97.65      1
110        1
70         1
64.8       1
56         1
130.6      1
447.2      1
60         1
21.6       1
Name: 태양광용량(kW), dtype: int64

In [32]:
building['ESS저장용량(kWh)'][building['ESS저장용량(kWh)']=='-']=0

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_29372\152263288.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  building['ESS저장용량(kWh)'][building['ESS저장용량(kWh)']=='-']=0


In [33]:
building['PCS용량(kW)'][building['PCS용량(kW)']=='-']=0

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_29372\3419152574.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  building['PCS용량(kW)'][building['PCS용량(kW)']=='-']=0


In [45]:
#building의 건물종류는 one-hot encoding해야할듯
building

,건물번호,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1,건물기타,110634.00,39570.00,0,0,0
1,2,건물기타,122233.47,99000.00,0,0,0
2,3,건물기타,171243.00,113950.00,40,0,0
3,4,건물기타,74312.98,34419.62,60,0,0
4,5,건물기타,205884.00,150000.00,0,2557,1000
...,...,...,...,...,...,...,...
95,96,호텔및리조트,93314.00,60500.00,0,0,0
96,97,호텔및리조트,55144.67,25880.00,0,0,0
97,98,호텔및리조트,53578.62,17373.75,0,0,0
98,99,호텔및리조트,53499.00,40636.00,0,0,0


In [46]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)
building_type = ohe.fit_transform(building[['건물유형']])
building_type

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [47]:
ohe.categories_

[array(['건물기타', '공공', '대학교', '데이터센터', '백화점및아울렛', '병원', '상용', '아파트', '연구소',
        '지식산업센터', '할인마트', '호텔및리조트'], dtype=object)]

In [48]:
pd.DataFrame(building_type, columns=[col for col in ohe.categories_[0]])

,건물기타,공공,대학교,데이터센터,백화점및아울렛,병원,상용,아파트,연구소,지식산업센터,할인마트,호텔및리조트
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [50]:
building_new = pd.concat([building.drop(columns=['건물유형']), pd.DataFrame(building_type, columns=[col for col in ohe.categories_[0]])], axis=1)

In [51]:
building_new

,건물번호,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW),건물기타,공공,대학교,데이터센터,백화점및아울렛,병원,상용,아파트,연구소,지식산업센터,할인마트,호텔및리조트
0,1,110634.00,39570.00,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,122233.47,99000.00,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,171243.00,113950.00,40,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,74312.98,34419.62,60,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,205884.00,150000.00,0,2557,1000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,93314.00,60500.00,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
96,97,55144.67,25880.00,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
97,98,53578.62,17373.75,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
98,99,53499.00,40636.00,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [52]:
test

,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%)
0,1,2022-08-25 00:00:00,23.5,0.0,2.2,72
1,1,2022-08-25 01:00:00,23.0,0.0,0.9,72
2,1,2022-08-25 02:00:00,22.7,0.0,1.5,75
3,1,2022-08-25 03:00:00,22.1,0.0,1.3,78
4,1,2022-08-25 04:00:00,21.8,0.0,1.0,77
...,...,...,...,...,...,...
16795,100,2022-08-31 19:00:00,22.5,0.0,0.9,84
16796,100,2022-08-31 20:00:00,20.7,0.0,0.4,95
16797,100,2022-08-31 21:00:00,20.2,0.0,0.4,98
16798,100,2022-08-31 22:00:00,20.1,0.0,1.1,97


In [35]:
#train 에는 풍속, 습도 데이터 존재
#test 에는 풍속 습도 데이터 없음
# 기온, 강수량, 풍속, 습도 -> 일조, 일사 예측
#